## Описание проекта
Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.


## Цель исследования
Осуществить предварительный анализ тарифов с целью проанализировать поведение клиентов и сделать вывод — какой тариф лучше.


## Описание тарифов

#### Тариф «Смарт»

    Ежемесячная плата: 550 рублей
    Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
    Стоимость услуг сверх тарифного пакета: 1. минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута); 2. сообщение: 3 рубля; 3. 1 Гб интернет-трафика: 200 рублей.

#### Тариф «Ультра»

    Ежемесячная плата: 1950 рублей
    Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
    Стоимость услуг сверх тарифного пакета: 1. минута разговора: 1 рубль; 2. сообщение: 1 рубль; 3. 1 Гб интернет-трафика: 150 рублей.

#### Примечание:
«Мегалайн» всегда округляет секунды до минут, а мегабайты — до гигабайт. Каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута.
Для веб-трафика отдельные сессии не считаются. Вместо этого общая сумма за месяц округляется в бо́льшую сторону. Если абонент использует 1025 мегабайт в этом месяце, с него возьмут плату за 2 гигабайта.

Описание данных
Таблица users (информация о пользователях):

    user_id — уникальный идентификатор пользователя
    first_name — имя пользователя
    last_name — фамилия пользователя
    age — возраст пользователя (годы)
    reg_date — дата подключения тарифа (день, месяц, год)
    churn_date — дата прекращения пользования тарифом (если значение пропущено, что тариф ещё действовал на момент выгрузки данных)
    city — город проживания пользователя
    tarif — название тарифного плана

Таблица calls (информация о звонках):

    id — уникальный номер звонка
    call_date — дата звонка
    duration — длительность звонка в минутах
    user_id — идентификатор пользователя, сделавшего звонок

Таблица messages (информация о сообщениях):

    id — уникальный номер сообщения
    message_date — дата сообщения
    user_id — идентификатор пользователя, отправившего сообщение

Таблица internet (информация об интернет-сессиях):

    id — уникальный номер сессии
    mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
    session_date — дата интернет-сессии
    user_id — идентификатор пользователя

Таблица tariffs (информация о тарифах):

    tariff_name — название тарифа
    rub_monthly_fee — ежемесячная абонентская плата в рублях
    minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
    messages_included — количество сообщений в месяц, включённых в абонентскую плату
    mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
    rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
    rub_per_message — стоимость отправки сообщения сверх тарифного пакета
    rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)

# Шаг 1. Предобработка данных

In [10]:
# импортнем наши любимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from math import ceil

% matplotlib inline

UsageError: Line magic function `%` not found.


In [11]:
# прочитаем данные из файлов
calls = pd.read_csv('../data/calls.csv')
internet = pd.read_csv('../data/internet.csv')
messages = pd.read_csv('../data/messages.csv')
tariffs = pd.read_csv('../data/tariffs.csv')
users = pd.read_csv('../data/users.csv')

# зададим датафреймам названия, для удобства работы с ними
calls.name = 'calls'
internet.name = 'internet'
messages.name = 'messages'
tariffs.name = 'tariffs'
users.name = 'users'

# создадим список датафреймов, для дальнейшей автоматизации работы с ними
data_list = [calls, internet, messages, tariffs, users]

# словарь для сериализации номеров месяцов в более удобный для представления вид
month_to_str = {
    1: 'Январь',
    2: 'Февраль',
    3: 'Март',
    4: 'Апрель',
    5: 'Май',
    6: 'Июнь',
    7: 'Июль',
    8: 'Август',
    9: 'Сентябрь',
    10: 'Октябрь',
    11: 'Ноябрь',
    12: 'Декабрь'
}

Проверим данные на наличие пропусков

In [12]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    # объявляем счетчик
    counter = -1
    display('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        display(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                display(f'  - в колонке {columns[counter]}: {i} пропусков')
    else:
        display(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# применим функцию ко всем датафреймам
for data in data_list:
    get_missing_values(data)

'============================================================'

'Отлично, в датафрейме calls отсутствуют пропуски.'

'============================================================'

'Отлично, в датафрейме internet отсутствуют пропуски.'

'============================================================'

...

'============================================================'

'Отлично, в датафрейме tariffs отсутствуют пропуски.'

'============================================================'

'В датафрейме users имеются следующие пропуски:'

'  - в колонке churn_date: 462 пропусков'

Пропуски есть только в колонке churn_date датафрейма users, что корректно, исходя их описания полученных данных.
Пропуски в этой колонке означают, что тариф ещё действовал на момент выгрузки данных.

In [13]:
def auto_change_dtypes(data: pd.DataFrame) -> None:
    """
    Автоматически определяет тип столбца, и изменяет его в соответствии с хранимыми значениями.
    Функция не возвращает новый датафрейм, а изменяет переданный в качестве аргумента.
    Функция заточена под данные конкретного проекта.
    Функция поддерживает автоматическое преобразование следующих типов и форматов данных:
     - int64
     - float64
     - str: если в названии колонки есть date и формат даты %Y-%m-%d,
            то переводит в формат pandas datetime, иначе - переводит строковые данные в нижний регистр

    Пример преобразования:
    data[column] является int64 и содержит значения в диапазоне от 0 до 100 - будет преобразован в int8
    data[date_column] является object и содержит в имени колонки date - будет преобразован в datetime

    :param data: pd.DataFrame
    :return: None
    """
    # получаем количество используемой датафреймом памяти
    memory_usage_before_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    # получаем описание датафрейма
    describe = data.describe()
    # получаем названия колонок
    columns = data.columns.to_list()
    # получаем типы данных
    dtypes = data.dtypes
    # количество типов данных
    indexes = len(dtypes)
    # создаем 2 словаря для int и float, содержащие в качестве ключей типы данных, а значений - список из min и max значений этих типов данных
    correct_int_dtypes = {'int8': [-2**7, 2**7-1], 'int16': [-2**15, 2**15-1], 'int32': [-2**31, 2**31-1]}
    correct_float_dtypes = {'float16': [-2.0**16, 2.0**16-1], 'float32': [-2.0**31, 2.0**31-1]}

    display(f'{"="*30} Работаем с датафреймом {data.name} {"="*30}')
    # пробегаем по индексам колонок датафрейма и типам данных колонок
    for index, dtype in zip(range(0, indexes), dtypes):

        # если тип int64, меняем на тип, соответствующий значениям колонок
        if dtype == np.int64:
            for key, value in zip(correct_int_dtypes.keys(), correct_int_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип float64, меняем на тип, соответствующий значениям колонок
        elif dtype == np.float64:
            for key, value in zip(correct_float_dtypes.keys(), correct_float_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип object и колонка содержит в названии 'date' - меняем на datetime
        elif dtype == object:
            if 'date' in columns[index]:
                display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на datetime')
                data[columns[index]] = pd.to_datetime(data[columns[index]], format='%Y-%m-%d')
            # иначе приводим данные к нижнему регистру
            else:
                data[columns[index]] = data[columns[index]].apply(lambda s: s.lower())

    # количество памяти, используемое датафреймом оптимизации типов данных
    memory_usage_after_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    bytes_in_mb = 2**23
    display(f'Использование памяти датафрейма до сжатия: {(memory_usage_before_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Использование памяти датафрейма после сжатия: {(memory_usage_after_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Сжато: {((memory_usage_before_change_dtypes - memory_usage_after_change_dtypes) / bytes_in_mb):.2f} мб.')

for data in data_list:
    auto_change_dtypes(data)

'============================== Работаем с датафреймом calls =============================='

'Изменяем тип колонки call_date датафрейма calls с object на datetime'

'Изменяем тип колонки duration датафрейма calls с float64 на float16'

'Изменяем тип колонки user_id датафрейма calls с int64 на int16'

'Использование памяти датафрейма до сжатия: 3.57 мб.'

...

'Изменяем тип колонки churn_date датафрейма users с object на datetime'

'Изменяем тип колонки reg_date датафрейма users с object на datetime'

'Использование памяти датафрейма до сжатия: 0.03 мб.'

'Использование памяти датафрейма после сжатия: 0.02 мб.'

'Сжато: 0.01 мб.'

В условии сказано, что пропущенные звонки имеют нулевую продолжительность. Для удобства работы с данными создадим дополнительный столбец в датафрейме calls, в котором будет значение True, если звонок был пропущен.

In [14]:
calls['is_missed'] = calls['duration'].apply(lambda x: True if x == 0 else False)
# проверим, что код отработал верно
calls.head()

,id,call_date,duration,user_id,is_missed
0,1000_0,2018-07-25,0.000000,1000,True
1,1000_1,2018-08-17,0.000000,1000,True
2,1000_2,2018-06-11,2.849609,1000,False
3,1000_3,2018-09-21,13.796875,1000,False
4,1000_4,2018-12-15,5.179688,1000,False


В датафрейме internet присутствует колонка Unnamed: 0, что говорит о том, что файл csv был сохранен вместе с индексами.
При загрузке датафрейма из файла, pandas создает новый индекс, а сохраненный в файле именуется 'Unnamed: 0'.
Исправим это.

In [15]:
internet.drop('Unnamed: 0', axis=1, inplace=True)
# проверим, что исправили ошибку
internet.head()

,id,mb_used,session_date,user_id
0,1000_0,112.9375,2018-11-25,1000
1,1000_1,1053.0000,2018-09-07,1000
2,1000_2,1197.0000,2018-06-25,1000
3,1000_3,550.5000,2018-08-22,1000
4,1000_4,302.5000,2018-09-24,1000


В условии сказано, что продолжительность каждого звонка округляется до минут в большую сторону, даже если он длился всего 1 секунду, будет засчитан как 1 минута.
Создадим столбец в датафрейме calls с округленными в большую сторону значениями продолжительности звонка.

In [16]:
calls['rounded_duration'] = calls['duration'].apply(lambda x: ceil(x))
# проверим, что код отработал верно
calls.head()

,id,call_date,duration,user_id,is_missed,rounded_duration
0,1000_0,2018-07-25,0.000000,1000,True,0
1,1000_1,2018-08-17,0.000000,1000,True,0
2,1000_2,2018-06-11,2.849609,1000,False,3
3,1000_3,2018-09-21,13.796875,1000,False,14
4,1000_4,2018-12-15,5.179688,1000,False,6


Посчитаем количество сделанных звонков и израсходованных минут разговора по месяцам.

In [17]:
# посчитаем количество сделанных звонков (не пропущенных) каждым пользователем по месяцам
calls_data = (
    calls
    .query("is_missed == False")
    .groupby(['user_id', calls['call_date'].dt.month]).count()
    .rename(month_to_str)
    .rename({'id': 'calls_count'}, axis=1)['calls_count']
    .reset_index()
)
calls_data.head()

,user_id,call_date,calls_count
0,1000,Май,17
1,1000,Июнь,28
2,1000,Июль,41
3,1000,Август,42
4,1000,Сентябрь,46


In [18]:
# посчитаем количество минут, израсходованных пользователем в месяц
# используем округленные данные, т.к. оператор всегда округляет секунды до минут в большую сторону
# при этом можем не исключать пропущенные, т.к. там нулевая продолжительность, которая не повлияет на сумму
duration_data = (
    calls
    .groupby(['user_id', calls['call_date'].dt.month])
    .agg({'rounded_duration': 'sum'})
    .rename(month_to_str)
    .reset_index()

)
duration_data.head()

,user_id,call_date,rounded_duration
0,1000,Май,159
1,1000,Июнь,172
2,1000,Июль,340
3,1000,Август,408
4,1000,Сентябрь,466


In [19]:
# посчитаем количество отправленных сообщений по месяцам
messages_data = (
    messages
    .groupby(['user_id', messages['message_date'].dt.month])
    .count()
    .rename(month_to_str)
    .rename({'id': 'messages_count'}, axis=1)['messages_count']
    .reset_index()
)

messages_data.head()

,user_id,message_date,messages_count
0,1000,Май,22
1,1000,Июнь,60
2,1000,Июль,75
3,1000,Август,81
4,1000,Сентябрь,57


In [20]:
# посчитаем объем израсходованного интернет трафика каждым пользователем, по месяцам
internet_traff_data = (
    internet
    .groupby(['user_id', internet['session_date'].dt.month])
    .sum()
    .rename(month_to_str)
    .reset_index()
)

# т.к. для веб-трафика отдельные сессии не считаются,
# а вместо этого оператором общая сумма за месяц округляется в бо́льшую сторону,
# то посчитаем количество ГБ, используемых пользователем и округлим полученное значение соответствующим образом
internet_traff_data['rounded_gb_used'] = (internet_traff_data['mb_used'] / 1024).apply(lambda x: ceil(x))

# проверим, что код отработал верно
internet_traff_data.head()

,user_id,session_date,mb_used,rounded_gb_used
0,1000,Май,2254.0,3
1,1000,Июнь,23232.0,23
2,1000,Июль,14008.0,14
3,1000,Август,14056.0,14
4,1000,Сентябрь,14568.0,15


Объединим полученные датафреймы

In [21]:
merged_data = pd.merge(duration_data.rename({'call_date': 'date'}, axis=1), messages_data.rename({'message_date': 'date'}, axis=1), on=['user_id', 'date'])

merged_data = pd.merge(merged_data, internet_traff_data.rename({'session_date': 'date'}, axis=1), on=['user_id', 'date'])

merged_data = pd.merge(merged_data, users[['user_id', 'tariff']], on='user_id')

merged_data.head()

,user_id,date,rounded_duration,messages_count,mb_used,rounded_gb_used,tariff
0,1000,Май,159,22,2254.0,3,ultra
1,1000,Июнь,172,60,23232.0,23,ultra
2,1000,Июль,340,75,14008.0,14,ultra
3,1000,Август,408,81,14056.0,14,ultra
4,1000,Сентябрь,466,57,14568.0,15,ultra


Посчитаем помесячную выручку с каждого пользователя.
Для этого вычтем бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика,
а остаток умножим на значение из тарифного плана и прибавим абонентскую плату, соответствующую тарифному плану.

In [22]:
def calculate_month_revenue(row: pd.Series) -> int:
    # получаем тарифные лимиты
    limits = tariffs.query("tariff_name == @row['tariff']")
    messages_limit = int(limits["messages_included"])
    mb_limit = int(limits["mb_per_month_included"])
    minutes_limit = int(limits["minutes_included"])
    month_payment = int(limits["rub_monthly_fee"])
    payment_per_gb = int(limits["rub_per_gb"])
    payment_per_message = int(limits["rub_per_message"])
    payment_per_minute = int(limits["rub_per_minute"])

    # прибавим к стоимости тарифа плату за перерасход смс
    if row["messages_count"] > messages_limit:
        month_payment += (row["messages_count"] - messages_limit) * payment_per_message

    # прибавим к стоимости тарифа плату за перерасход гб интернетов
    if row["rounded_gb_used"] > (mb_limit / 1024):
        month_payment += ceil(row["rounded_gb_used"] - mb_limit / 1024) * payment_per_gb

    # прибавим к стоимости тарифа плату за перерасход минут
    if row["rounded_duration"] > minutes_limit:
        month_payment += (row["rounded_duration"] - minutes_limit) * payment_per_minute

    return month_payment

# посчитаем месячную выручку с каждого пользователя
merged_data["month_revenue"] = merged_data.apply(calculate_month_revenue, axis=1)

merged_data.head()

,user_id,date,rounded_duration,messages_count,mb_used,rounded_gb_used,tariff,month_revenue
0,1000,Май,159,22,2254.0,3,ultra,1950
1,1000,Июнь,172,60,23232.0,23,ultra,1950
2,1000,Июль,340,75,14008.0,14,ultra,1950
3,1000,Август,408,81,14056.0,14,ultra,1950
4,1000,Сентябрь,466,57,14568.0,15,ultra,1950


## ШАГ 3.

Опишите поведение клиентов оператора, исходя из выборки.
Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц?
Посчитайте среднее количество, дисперсию и стандартное отклонение.
Постройте гистограммы.
Опишите распределения.

Видим, что каждый месяц количество звонков стремительно увеличивается.
Посчитаем количество отправленных сообщений по месяцам.

In [189]:
columns = {
    'rounded_duration': 'В среднем пользователю требуется _минут',
    'messages_count': 'В среднем пользователю требуется _сообщений',
    'rounded_gb_used': 'В среднем пользователю требуется _Гб'
}

tariffs = ['ultra', 'smart']

means = merged_data.groupby('tariff').mean().reset_index()

for tariff in tariffs:
    display(f"{'=' * 20} {tariff} {'=' * 20}")
    for column, text in zip(columns.keys(), columns.values()):
        text1, text2 = text.split('_')
        display(f"{text1}{int(means[means['tariff'] == tariff][column])} {text2}")


'==================== ultra ===================='

'В среднем пользователю требуется 556 минут'

'В среднем пользователю требуется 61 сообщений'

'В среднем пользователю требуется 19 Гб'

'==================== smart ===================='

'В среднем пользователю требуется 421 минут'

'В среднем пользователю требуется 38 сообщений'

'В среднем пользователю требуется 16 Гб'

In [194]:
# построим распределение потребностей пользователей тарифов по месяцам

columns = {
    'rounded_duration': 'Время звонка (мин)',
    'messages_count': 'Количество сообщений',
    'rounded_gb_used': 'Использование интернета (гб)'
}

for column, title in zip(columns.keys(), columns.values()):
    fig = px.box(merged_data, x="date", y=column, color="tariff")
    fig.update_layout(
        showlegend=True,
        xaxis_title="Месяц",
        yaxis_title=title
    )

    fig.show()

In [205]:
columns = {
    'rounded_duration': 'Время звонка (мин)',
    'messages_count': 'Количество сообщений',
    'rounded_gb_used': 'Использование интернета (гб)'
}

for column, title in zip(columns.keys(), columns.values()):
    fig = px.histogram(merged_data, x=column, color='tariff', opacity=0.7, text_auto=True)
    fig.update_layout(
        showlegend=True,
        xaxis_title=title,
        yaxis_title="Количество пользователей"
    )

    fig.show()


In [141]:
merged_data.pivot_table(index=['tariff', 'date'], values=['rounded_duration', 'rounded_gb_used', 'messages_count'], aggfunc=['sum', 'mean'])

sum                                   \
                messages_count rounded_duration rounded_gb_used   
tariff date                                                       
smart  Август             7887            86751            3426   
       Апрель             3443            36633            1326   
       Декабрь           12095           135578            5145   
       Июль               6784            71222            2774   
       Июнь               5430            56284            2247   
       Май                4762            48752            1982   
       Март               2453            27123            1020   
       Ноябрь            10748           118423            4597   
       Октябрь            9906           108130            4271   
       Сентябрь           8714            95268            3652   
       Февраль            1397            15587             608   
       Январь              620             6531             275   
ultra  Август             4889            43782            1611   
       Апрель             1727            18993             644   
       Декабрь            8102            71021            2437   
       Июль               4229            38541            1392   
       Июнь               3206            31359            1166   
       Май                2402            25542             909   
       Март               1195            13822             429   
       Ноябрь             7195            59951            2114   
       Октябрь            6864            55978            1991   
       Сентябрь           5719            50836            1747   
       Февраль             522             5600             209   
       Январь              304             3332              98   

                          mean                                   
                messages_count rounded_duration rounded_gb_used  
tariff date                                                      
smart  Август        39.044554       429.460396       16.960396  
       Апрель        35.864583       381.593750       13.812500  
       Декабрь       42.290210       474.048951       17.989510  
       Июль          38.988506       409.321839       15.942529  
       Июнь          38.510638       399.177305       15.936170  
       Май           39.032787       399.606557       16.245902  
       Март          35.550725       393.086957       14.782609  
       Ноябрь        39.955390       440.234201       17.089219  
       Октябрь       39.154150       427.391304       16.881423  
       Сентябрь      39.252252       429.135135       16.450450  
       Февраль       27.940000       311.740000       12.160000  
       Январь        20.000000       210.677419        8.870968  
ultra  Август        58.903614       527.493976       19.409639  
       Апрель        45.447368       499.815789       16.947368  
       Декабрь       72.990991       639.828829       21.954955  
       Июль          61.289855       558.565217       20.173913  
       Июнь          50.888889       497.761905       18.507937  
       Май           50.041667       532.125000       18.937500  
       Март          47.800000       552.880000       17.160000  
       Ноябрь        67.242991       560.289720       19.757009  
       Октябрь       70.762887       577.092784       20.525773  
       Сентябрь      63.544444       564.844444       19.411111  
       Февраль       37.285714       400.000000       14.928571  
       Январь        43.428571       476.000000       14.000000

In [153]:
s = merged_data.groupby(['tariff', 'date']).mean().reset_index()
fig = px.bar(s,
             x='date',
             y='rounded_duration',
             color='tariff',
             text_auto=True,
             title="Среднее время вызовов")

fig.update_layout(
    showlegend=True,
    xaxis_title="Месяц",
    yaxis_title="Среднее время вызова"
)

fig.show()

Количество сообщений пользователей также стремительно растет.

In [7]:
users.head()

NameError: name 'users' is not defined

## Шаг 2. Анализ